In [1]:
import torch
from ultralytics import YOLO
from tkinter import Tk, Button, Label, filedialog, StringVar, OptionMenu, messagebox
from PIL import Image, ImageTk
import cv2

# Global variables
model = YOLO('yolov8n.pt')  # Default model
image_path = None

# Function to perform object detection
def detect_objects():
    global image_path, model

    if not image_path:
        messagebox.showerror("Error", "Please upload an image first.")
        return

    # Perform inference on the image
    results = model(image_path)

    # Get the annotated image with bounding boxes and labels
    annotated_image = results[0].plot()

    # Count the number of detected objects
    num_objects = len(results[0].boxes)
    cv2.putText(annotated_image, f'Objects: {num_objects}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Convert the annotated image to a format suitable for Tkinter
    annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(annotated_image)
    img = ImageTk.PhotoImage(image=img)

    # Display the annotated image in the GUI
    image_label.config(image=img)
    image_label.image = img

    # Update status
    status_label.config(text=f"Status: Detection complete. Objects detected: {num_objects}")

# Function to upload an image
def upload_image():
    global image_path
    image_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if image_path:
        # Display the uploaded image
        img = Image.open(image_path)
        img = img.resize((400, 400))  # Resize for display
        img = ImageTk.PhotoImage(image=img)
        image_label.config(image=img)
        image_label.image = img
        status_label.config(text="Status: Image uploaded.")

# Function to save the annotated image
def save_image():
    if not image_path:
        messagebox.showerror("Error", "No image to save.")
        return

    save_path = filedialog.asksaveasfilename(defaultextension=".jpg", filetypes=[("JPEG files", "*.jpg"), ("PNG files", "*.png")])
    if save_path:
        # Perform inference and save the annotated image
        results = model(image_path)
        results[0].save(save_path)
        messagebox.showinfo("Success", f"Image saved to {save_path}")

# Function to change the model
def change_model(*args):
    global model
    model = YOLO(model_var.get())

# Create the main GUI window
root = Tk()
root.title("YOLOv8 Image Object Detection")
root.geometry("800x600")  # Larger window size

# Model selection dropdown
model_var = StringVar(value="yolov8n.pt")
model_options = ["yolov8n.pt", "yolov8s.pt", "yolov8m.pt", "yolov8l.pt", "yolov8x.pt"]
model_menu = OptionMenu(root, model_var, *model_options, command=change_model)
model_menu.pack(pady=10)

# Button to upload an image
Button(root, text="Upload Image", command=upload_image).pack(pady=10)

# Label to display the image
image_label = Label(root)
image_label.pack(pady=10)

# Button to perform object detection
Button(root, text="Detect Objects", command=detect_objects).pack(pady=10)

# Button to save the annotated image
Button(root, text="Save Result", command=save_image).pack(pady=10)

# Status label
status_label = Label(root, text="Status: Waiting for image upload.", font=("Arial", 12))
status_label.pack(pady=10)

# Run the GUI main loop
root.mainloop()


image 1/1 C:\Users\hp\Desktop\images_sample.jpg: 480x640 3 persons, 1 tie, 1 pizza, 1 laptop, 125.7ms
Speed: 0.0ms preprocess, 125.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
